In [21]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler
from IPython.display import display
from IPython.core.debugger import set_trace

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. 초기화

In [22]:
crawler = NewsCrawler(storage='bigquery')

checking bigquery storage... done (27.82 seconds)
we have total 324924 articles (301806 downloaded, 23118 trashed)


# 1. URL을 모은다

In [18]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [23]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (81.11 seconds): Collecting URLs... yahoo               

100.00% (0.12 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,25168,25091,786
dailymail,2817,2817,137
reuters,1109,1109,115
investing.com,967,967,97
chinadaily,679,679,80
...,...,...,...
nationalreview,721,719,0
news24,1224,1221,0
techcrunch,19,19,0
arstechnica,208,208,0


,url,pubs,actual_pub
0,https://politico.com/magazine/story/2019/10/23...,"politico, realclearpolitics",politico
1,https://politico.com/magazine/story/2019/10/23...,"politico, realclearpolitics",politico
2,https://politico.com/magazine/story/2019/10/21...,"politico, realclearpolitics",politico
3,https://politico.com/news/2019/10/30/latinos-b...,"politico, realclearpolitics",politico
4,https://politico.com/news/2019/10/30/pete-butt...,"politico, realclearpolitics",politico
...,...,...,...
72,https://nbcnews.com/think/opinion/trump-offici...,"msnbc, nbcnews",nbcnews
73,https://nbcnews.com/business/economy/federal-r...,"msnbc, nbcnews",nbcnews
74,https://nbcnews.com/news/latino/democrat-accus...,"msnbc, nbcnews",nbcnews
75,https://nbcnews.com/news/us-news/iowa-man-accu...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 크롤링한다

In [24]:
crawl_result = crawler.crawl(); crawl_result

6.87% (14.97 seconds): Crawling... investing.com       

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100.00% (290.45 seconds): Crawling... dailymail           

Recording to bigquery storage... done


collect                                crawl              
               collected collected_unique selected downloaded trashed total
all                25168            25091      786        465     321   786
dailymail           2817             2817      137        128       9   137
reuters             1109             1109      115         51      64   115
investing.com        967              967       97         22      75    97
chinadaily           679              679       80         33      47    80
...                  ...              ...      ...        ...     ...   ...
nationalreview       721              719        0          0       0     0
news24              1224             1221        0          0       0     0
techcrunch            19               19        0          0       0     0
arstechnica          208              208        0          0       0     0
cryptonews           326              326        0          0       0     0

[74 rows x 6 columns]

In [27]:
23118+321

23439

Bigquery와 local 뉴스 갯수가 다를 것이다. Bigquery에 뉴스를 모두 업데이트 한 후 해당 repository에 문제가 생겨서, github에서 전일기준 데이터를 clone하여, 뉴스를 다시 다운로드 받은 날이 있다. 2019-10-22

In [28]:
from pathlib import Path
downloaded = [p.stem for p in Path('newsdata/downloaded').glob('**/*.json')]
trashed = [p.stem for p in Path('newsdata/trashed').glob('**/*.json')]

In [29]:
len(downloaded)

281334

In [19]:
fnames_remove = set(downloaded) & set(trashed)

In [21]:
for fname in fnames_remove:
    file = Path('newsdata/trashed/' + fname[:3] + '/' + fname + '.json')
    file.unlink()

In [8]:
import pandas_gbq as gbq
from google.oauth2 import service_account

proj = 'global-news-crawl'
table_downloaded = 'news_dataset.downloaded'
table_trashed = 'news_dataset.trashed'
credentials = service_account.Credentials.from_service_account_file('global-news-crawl-c48d7cd9aa81.json')

In [16]:
%%time
qry = 'SELECT * FROM `{}` WHERE published_at="2019-10-23"'.format(proj + '.' + table_downloaded)
articles = gbq.read_gbq(qry, project_id=proj, credentials=credentials)#, use_bqstorage_api=True)

Wall time: 14.7 s


In [13]:
articles

,id,pub,url,downloaded_at,title,language,text,description,authors,top_image,published_at
0,a4632835bc5b202d969eddac2469ac6956b59d05,telegraph,https://telegraph.co.uk/luxury/design/super-ri...,2019-10-30 08:05:20.835619+00:00,How the super-rich decorate a £1m nursery: fro...,en,Having a baby and not started on the nursery y...,Having a baby and not started on the nursery yet?,Lucy Scott,https://www.telegraph.co.uk/content/dam/luxury...,2019-10-23
1,ea949c84d1f0691c3cda34abd0f08ade86683aca,wsj,https://wsj.com/articles/auto-sectors-struggle...,2019-10-29 23:57:06.161380+00:00,‘Peak Car’ Is Holding Back the Global Economy,en,"At this week’s biennial Tokyo Motor Show, Toyo...",The global auto industry is going through a sh...,Greg Ip,https://images.wsj.net/im-119723/social,2019-10-23
2,f9d9cc040d2d8521a99c373bcb173203b9223dc8,reuters,https://reuters.com/news/archive/innovationint...,2019-10-29 02:37:07.610497+00:00,Innovation and Intellectual Property News Head...,en,"Bezos's Blue Origin partners with Lockheed, ot...",Get the latest breaking news and top news head...,Reuters Editorial,https://s4.reutersmedia.net/resources_v2/image...,2019-10-23
3,c972de109ece19caa5cb7825ce9b39006b1d1fb8,telegraph,https://telegraph.co.uk/beauty/skin/18-vegan-s...,2019-10-28 23:34:35.114853+00:00,The £18 vegan 'super cream' that promises to r...,en,Touted as the best thing you can do for the en...,Touted as the best thing you can do for the en...,"Amerley Ollennu, Krissy Turner",https://www.telegraph.co.uk/content/dam/beauty...,2019-10-23
4,fd0a7185bd7fa8a4ba31afa73ed728de1e2f3951,news.com.au,https://realestate.com.au/news/superyachts-epi...,2019-10-29 00:18:24.427543+00:00,Superyacht's epic interiors you just have to s...,en,"Sleek, spectacular and overtly opulent, this n...","Sleek, spectacular and overtly opulent, this n...",Nicole Cox,https://www.realestate.com.au/blog/wp-content/...,2019-10-23
...,...,...,...,...,...,...,...,...,...,...,...
6898,29e38d0fdd41e3fbc7d73c911a2f2de6e7461315,realclearpolitics,https://americanthinker.com/articles/2019/10/w...,2019-10-23 08:32:16.341598+00:00,Will Abortion Sink the Democratic Party in 2020?,en,The big wild card issue heading into the 2020 ...,None,None,https://www.americanthinker.com/images/bucket/...,2019-10-23
6899,35f3e2c53eb66251a11289b4e58e347ed5efc673,realclearpolitics,https://realcleardefense.com/articles/2019/10/...,2019-10-23 14:55:39.729652+00:00,China’s Own ‘Great Delusion’,en,China has turned American hubris on its head –...,China has turned American hubris on its head –...,None,https://assets.realclear.com/images/49/492031_...,2019-10-23
6900,29eb14212ef903fe10cd0c8c8b18fa248761219b,americanconservative,https://theamericanconservative.com/larison/mo...,2019-10-23 15:54:57.745061+00:00,Most Americans Blame Trump for the Crisis with...,en,A survey conducted for the Brookings Instituti...,None,Daniel Larison,https://www.theamericanconservative.com/wp-con...,2019-10-23
6901,34f38e00ebf6d4fddcfa13a6bd3819c4b09ec554,americanconservative,https://theamericanconservative.com/larison/a-...,2019-10-23 14:55:26.490251+00:00,A Deadly Fuel Shortage in Yemen Threatens 15 M...,en,Oxfam warns that 15 million Yemenis are being ...,None,Daniel Larison,https://www.theamericanconservative.com/wp-con...,2019-10-23
